# ini文件读取
---

### 优点：</p>
    ini文件的存储格式简单、便于手动修改。适合存储一些变量参数。
### 缺点：
##### 1.保存的数据都是字符串，bool类型读取后需要转换
##### 2.编码问题：
    当ini文件重新使用编码UTF-8或者unicode存储后，会在文件开头自动添加字符“\xff\xfe”或“\xef\xbb\xbf”（文件中看不到的），导致读取出文件时会碰到如下报错：“ConfigParser.MissingSectionHeaderError: File contains no section headers.”解决方案见下文。
    解决方案1：

___

## 一、ini文件格式

<img style="float: left;" src="./file/format.jpg" width="25%">

## 二、文件的读取

### 解决编码问题的方法思路：
#### 解决方案1：
        使用txt打开文件，重新保存，编码格式改为asicii。这样程序可以正常读取，但是你打开文档查看是乱码....
#### 解决方案2（适用于经常性手动修改ini文件的场景）：
        remove_BOM函数读取ini文件，去掉文件开头的无用字符。然后保存为另外一个ascii编码的文件用来读取。
        也就是说一个文件可以给使用者来查看与修改。程序读取时将文件另存为ascii编码专门用来读取。这样既能解决显示问题，又能解决读取问题，一举两得。

In [1]:
def remove_BOM(config_path):#去掉配置文件开头的BOM字节
    content = open(config_path).read()#,encoding='utf-8'
    content = re.sub(r"\xfe\xff","", content)
    content = re.sub(r"\xff\xfe","", content)
    content = re.sub(r"\xef\xbb\xbf","", content)
    open('read.ini', 'w').write(content)

def read_config(file='config.ini'):
    remove_BOM(file)#去除文件开头字符并另存为read.ini
    cfg = ConfigParser()
    try:
        cfg.read('read.ini')#读取read.ini
        n = cfg.get('user', 'name')#读取key中的键值
        d = cfg.get('list', 'description')
        de = cfg.get('mode', 'demo')
        st=cfg.get('mode','chromeState')
        if de == 'False':
            de = False
        else:
            de = True
        return [n, d, de, st]
    except Exception as e:
        #print(e)
        raise Exception("\033[0;31;mRead false! Please check file config.ini !\033[0;30;m")

## 写文件 

In [2]:

import configparser

config = configparser.ConfigParser()
config['DEFAULT'] = {'ServerAliveInterval': '45',
                    'Compression': 'yes',
                    'Compression': 'yes',
                    'CompressionLevel': '9'}
config['bitbucket.org'] = {}
config['bitbucket.org']['User'] = 'hg'
config['topsecret.server.com'] = {}
topsecret = config['topsecret.server.com']
topsecret['Port'] = '50022'     # mutates the parser
topsecret['ForwardX11'] = 'no'  # same here
config['DEFAULT']['ForwardX11'] = 'yes'
with open('configTest.ini', 'w') as configfile:
    config.write(configfile)